##installing dependencies

In [1]:
!pip install google-generativeai firebase-admin dateparser


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 12.8 MB/s eta 0:00:00


##setting up gemini api

In [3]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 8.4 MB/s eta 0:00:00


In [4]:
import google.generativeai as genai
from google.colab import userdata

# Fetch the key and configure
api_key = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=api_key)

##We define the instructions for the model

In [7]:

system_instruction = """
You are a database assistant for a Class Booking App.
The database has a collection called 'classes' with these fields:
- name (string)
- instructor (string)
- category (string: 'Yoga', 'HIIT', 'Dance')
- capacity (number)
- date (string: YYYY-MM-DD)

Your task: Convert user questions into a JSON object for a Firestore query.
Output ONLY the JSON. No conversational text.

Example 1: "Show me yoga classes for tomorrow"
Output: {"field": "category", "operator": "==", "value": "Yoga"}

Example 2: "Is there any HIIT class with more than 5 spots?"
Output: {"field": "capacity", "operator": ">", "value": 5}
"""

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    system_instruction=system_instruction
)

In [8]:
def get_db_query(user_input):
    response = model.generate_content(user_input)
    return response.text

print(get_db_query("Are there any dance classes?"))

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
Please retry in 48.700007812s.

In [9]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-prev

In [11]:
import google.generativeai as genai
import time

# Setup
api_key = "YOUR_NEW_API_KEY" # Replace this
genai.configure(api_key=api_key)

# The instructions for the model
system_instruction = """
You are a Class Booking Assistant.
Convert user requests into a JSON query for Firestore.
Fields: name (str), category (str), slots (int), date (YYYY-MM-DD).

Return ONLY JSON. Example: {"field": "category", "op": "==", "val": "Yoga"}
"""

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash", # Use the stable 2.0 version
    system_instruction=system_instruction
)

def get_db_query(user_input):
    try:
        response = model.generate_content(user_input)
        return response.text
    except Exception as e:
        if "429" in str(e):
            print("Quota hit! Waiting 10 seconds...")
            time.sleep(10) # Simple retry logic
            return get_db_query(user_input)
        return f"Error: {e}"

# Test it
print(get_db_query("Show me Yoga classes"))

Error: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.


In [13]:
import google.generativeai as genai
from google.colab import userdata

# 1. Setup with error handling
try:
    API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=API_KEY)
    print("✅ Configuration successful")
except Exception as e:
    print(f"❌ Setup failed: {e}")

# 2. Test connectivity with a basic model first
try:
    # We use 1.5-flash as it is the most stable free tier model
    model_test = genai.GenerativeModel('gemini-2.0-flash')
    response = model_test.generate_content("Hello!")
    print(f"✅ Connection verified: {response.text[:10]}...")
except Exception as e:
    print(f"❌ Model check failed: {e}")

# 3. Your specific Hackathon Logic
system_instruction = "You are a DB assistant. Return ONLY JSON: {'field': 'x', 'op': '==', 'val': 'y'}"
db_model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=system_instruction
)

def get_query(text):
    return db_model.generate_content(text).text

# Try it
print(f"DB Query Result: {get_query('Find yoga classes')}")

✅ Configuration successful
✅ Connection verified: Hi there! ...


NotFound: 404 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [14]:
import google.generativeai as genai
from google.colab import userdata

# 1. Setup
API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=API_KEY)

# 2. Recommended model for Hackathons (Fast & Stable)
# Options: 'gemini-2.0-flash' (Stable) or 'gemini-3-flash-preview' (Latest)
MODEL_ID = 'gemini-2.0-flash'

system_instruction = """
You are a Database Query Generator for a Flutter app.
User asks about class bookings. You output ONLY JSON.
Schema: {"field": string, "operator": "==" | ">" | "<", "value": any}
"""

try:
    model = genai.GenerativeModel(
        model_name=MODEL_ID,
        system_instruction=system_instruction
    )

    # Test it
    response = model.generate_content("Are there any Yoga classes?")
    print(f"✅ Success! Generated Query: {response.text}")

except Exception as e:
    print(f"❌ Failed with {MODEL_ID}. Error: {e}")
    print("\nTip: Run [m.name for m in genai.list_models()] to see your active models.")

✅ Success! Generated Query: ```json
[
  {"field": "className", "operator": "==", "value": "Yoga"}
]
```
